In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
import time
import sys

### Les SVM sont des modèles connus pour être bons. Ils ont tout de fois de nombreux inconvénients, en particulier pour nous : au dela de 10000 samples, le temps de calcul devient long, et on ne peut obtenir une prédiction en probabilités qu'au prix d'un couteux mécanisme de cross-validation interne au classifieur. Malgré cela, essayons !

In [23]:
train_fname = '../data/train.csv'
test_fname = '../data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
f_con = ['NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB',
        'INV_NB_PAYS','INV_NB_TYPE','cited_n','cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'cited_age_std','NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
        'NB_BACKWARD','pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_ROOT_CLASSES','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC']
(n_samples,n_variables) = (df.shape[0],len(f_con))

X_train = df[f_con].values
y_train = df.VARIABLE_CIBLE == 'GRANTED'
X_test = df_test[f_con].values
df = df.reindex(np.random.permutation(df.index))
X_train2 = df[f_con].values
y_train2 = df.VARIABLE_CIBLE == 'GRANTED'
imputer = Imputer()
# Imputer permet de combler les trous quand des données manquent. Par défaut il prend la moyenne de la dernière donnée vue
# et de la prochaine. Ces données ne sont donc pas normalisées.
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)
X_train2 = imputer.fit_transform(X_train2)
# Normalisation des features.
scale(X_train,copy=False);
scale(X_test,copy=False);
scale(X_train2,copy=False);

In [67]:
n_threads = -1
n_taken = 100
clf = SVC(C = 0.55,kernel='rbf')
clf.fit(X_train2[0:n_taken],y_train2[0:n_taken])
y_pred_train = clf.predict(X_train)
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

Score (optimiste) sur le train : 0.5132358517


In [66]:
# 0.5132 kernel = 'rbf' C = 0.54

## Ca n'a pas conduit à grand chose...